In [21]:
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import nltk
import re
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tag import PerceptronTagger
from nltk.data import find
from tqdm import tqdm
import itertools

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/wuga/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
df = pd.read_json('data/video/Amazon_Instant_Video_5.json', lines=True)

In [5]:
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,B000H00VBQ,"[0, 0]",2,I had big expectations because I love English ...,"05 3, 2014",A11N155CW1UV02,AdrianaM,A little bit boring for me,1399075200
1,B000H00VBQ,"[0, 0]",5,I highly recommend this series. It is a must f...,"09 3, 2012",A3BC8O2KCL29V2,Carol T,Excellent Grown Up TV,1346630400
2,B000H00VBQ,"[0, 1]",1,This one is a real snoozer. Don't believe anyt...,"10 16, 2013",A60D5HQFOTSOM,"Daniel Cooper ""dancoopermedia""",Way too boring for me,1381881600
3,B000H00VBQ,"[0, 0]",4,Mysteries are interesting. The tension betwee...,"10 30, 2013",A1RJPIGRSNX4PW,"J. Kaplan ""JJ""",Robson Green is mesmerizing,1383091200
4,B000H00VBQ,"[1, 1]",5,"This show always is excellent, as far as briti...","02 11, 2009",A16XRPF40679KG,Michael Dobey,Robson green and great writing,1234310400


In [6]:
df = df.drop(['reviewTime','helpful','reviewerName','summary'], axis=1)

In [7]:
df.head()

,asin,overall,reviewText,reviewerID,unixReviewTime
0,B000H00VBQ,2,I had big expectations because I love English ...,A11N155CW1UV02,1399075200
1,B000H00VBQ,5,I highly recommend this series. It is a must f...,A3BC8O2KCL29V2,1346630400
2,B000H00VBQ,1,This one is a real snoozer. Don't believe anyt...,A60D5HQFOTSOM,1381881600
3,B000H00VBQ,4,Mysteries are interesting. The tension betwee...,A1RJPIGRSNX4PW,1383091200
4,B000H00VBQ,5,"This show always is excellent, as far as briti...",A16XRPF40679KG,1234310400


In [8]:
stop_words = stopwords.words('english')

In [38]:
#df['tmp'] = df['reviewText'].apply(lambda x: [item for item in x.lower().split() if item not in stop_words]+[' '])
df['tmp'] = df['reviewText'].apply(lambda x: nltk.word_tokenize(x.lower())+[' '])

In [39]:
clean = df['tmp'].tolist()

In [12]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [13]:
text = df['reviewText'].tolist()

In [14]:
t = Tokenizer()
t.fit_on_texts(text)

In [15]:
print(len(t.word_counts))

59638


In [16]:
df['vector'] = t.texts_to_sequences(text)

In [40]:
df.head()

,asin,overall,reviewText,reviewerID,unixReviewTime,tmp,vector
0,B000H00VBQ,2,I had big expectations because I love English ...,A11N155CW1UV02,1399075200,"[I, had, big, expectations, because, I, love, ...","[7, 87, 246, 1464, 101, 7, 55, 778, 114, 10, 9..."
1,B000H00VBQ,5,I highly recommend this series. It is a must f...,A3BC8O2KCL29V2,1346630400,"[I, highly, recommend, this, series, ., It, is...","[7, 347, 237, 9, 30, 8, 6, 3, 267, 12, 327, 41..."
2,B000H00VBQ,1,This one is a real snoozer. Don't believe anyt...,A60D5HQFOTSOM,1381881600,"[This, one, is, a, real, snoozer, ., Do, n't, ...","[9, 27, 6, 3, 150, 17795, 96, 324, 285, 20, 36..."
3,B000H00VBQ,4,Mysteries are interesting. The tension betwee...,A1RJPIGRSNX4PW,1383091200,"[Mysteries, are, interesting, ., The, tension,...","[1048, 19, 106, 1, 893, 195, 14987, 2, 1, 4029..."
4,B000H00VBQ,5,"This show always is excellent, as far as briti...",A16XRPF40679KG,1234310400,"[This, show, always, is, excellent, ,, as, far...","[9, 15, 143, 6, 230, 16, 187, 16, 584, 385, 46..."


In [18]:
reverse_word_map = dict(map(reversed, t.word_index.items()))

In [19]:
reverse_word_map[7]

u'i'

In [41]:
import nltk

clean=itertools.chain.from_iterable(clean)

bigrams = nltk.collocations.BigramAssocMeasures()
trigrams = nltk.collocations.TrigramAssocMeasures()

bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(clean)
trigramFinder = nltk.collocations.TrigramCollocationFinder.from_words(clean)

In [42]:
#bigrams
bigram_freq = bigramFinder.ngram_fd.items()
bigramFreqTable = pd.DataFrame(list(bigram_freq), columns=['bigram','freq']).sort_values(by='freq', ascending=False)

#trigrams
trigram_freq = trigramFinder.ngram_fd.items()
trigramFreqTable = pd.DataFrame(list(trigram_freq), columns=['trigram','freq']).sort_values(by='freq', ascending=False)

In [43]:
#get english stopwords
en_stopwords = set(stopwords.words('english'))

#function to filter for ADJ/NN bigrams
def rightTypes(ngram):
    if '-pron-' in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in en_stopwords or word.isspace():
            return False
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False

#filter bigrams
filtered_bi = bigramFreqTable[bigramFreqTable.bigram.map(lambda x: rightTypes(x))]

In [48]:
filtered_bi.head(30)

,bigram,freq
673747,"(story, line)",1130
578240,"(second, season)",692
751958,"(great, show)",676
410704,"(story, lines)",597
94298,"(character, development)",473
577659,"(next, season)",472
507550,"(special, effects)",438
1006,"(main, characters)",421
135447,"(good, show)",414
554159,"(Amazon, Prime)",411


In [45]:
#filter for only those with more than 20 occurences
bigramFinder.apply_freq_filter(20)
trigramFinder.apply_freq_filter(20)

bigramPMITable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.pmi)), columns=['bigram','PMI']).sort_values(by='PMI', ascending=False)

In [47]:
bigramPMITable.head(30)

,bigram,PMI
0,"(BATTLESTAR, GALACTICA)",17.463650
1,"(Yankee, Pedlar)",17.168194
2,"(Olga, Kurylenko)",17.136486
3,"(Rufus, Sewell)",16.995433
4,"(Arrested, Development)",16.923082
5,"(Ty, Burrell)",16.923082
6,"(Lark, Rise)",16.858952
7,"(roller, coaster)",16.822044
8,"(Bebe, Neuwirth)",16.701810
9,"(Aldis, Hodge)",16.584957


In [50]:
bigramPMITable['Phrases'] = bigramPMITable['bigram'].apply(lambda x: ' '.join(x))

In [52]:
bigramPMITable.drop(['bigram'], axis=1).head()

,PMI,Phrases
0,17.463650,BATTLESTAR GALACTICA
1,17.168194,Yankee Pedlar
2,17.136486,Olga Kurylenko
3,16.995433,Rufus Sewell
4,16.923082,Arrested Development
